# 第二课：Prompt Template 和 Chain

## 学习目标
1. 了解 Prompt Template（提示词模板）
2. 了解 Chain（链）的概念
3. 使用 LCEL (LangChain Expression Language) 构建工作流

## 为什么需要这些概念？

- **Prompt Template**：让提示词可复用、可参数化
- **Chain**：将多个处理步骤连接起来，形成流水线

## 准备工作

In [ ]:
import os
from langchain_openai import ChatOpenAI

# 设置 API Key（替换为你的 Key）
os.environ["DASHSCOPE_API_KEY"] = "your_api_key_here"

def create_qwen_chat():
    """创建千问聊天模型"""
    api_key = os.getenv("DASHSCOPE_API_KEY")
    if not api_key or api_key == "your_api_key_here":
        raise ValueError("⚠️ 请设置你的 DASHSCOPE_API_KEY")
    
    return ChatOpenAI(
        model="qwen-plus",
        openai_api_key=api_key,
        openai_api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
        temperature=0.7,
    )

print("✅ 准备完成")

---
## 1. Prompt Template（提示词模板）

Prompt Template 让你可以：
- 定义可复用的提示词结构
- 动态插入变量
- 保持提示词的一致性

### 基础模板

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 创建一个简单的提示词模板
# {topic} 是一个变量占位符
prompt = ChatPromptTemplate.from_template(
    "你是一位资深的技术专家。请用通俗易懂的语言解释什么是 {topic}，"
    "并给出一个实际应用的例子。回答控制在100字以内。"
)

# 查看模板的变量
print(f"模板变量: {prompt.input_variables}")

# 格式化模板 - 将变量替换为实际值
formatted = prompt.format(topic="机器学习")
print(f"\n格式化后的提示词:\n{formatted}")

In [ ]:
# 使用模板调用模型
chat = create_qwen_chat()

# format_messages 返回消息列表
messages = prompt.format_messages(topic="机器学习")
response = chat.invoke(messages)

print("问题: 什么是机器学习？")
print(f"\n回复:\n{response.content}")

In [ ]:
# 复用同一个模板，解释不同的概念
topics = ["区块链", "云计算", "人工智能"]

for topic in topics:
    response = chat.invoke(prompt.format_messages(topic=topic))
    print(f"\n📚 {topic}:")
    print(f"{response.content}")
    print("-" * 50)

### 聊天提示词模板

`ChatPromptTemplate` 支持多种消息类型：
- `system`: 系统消息
- `human`: 用户消息
- `ai`: AI 消息

In [ ]:
# 创建带有系统消息的聊天模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个{role}，专门帮助用户解答{domain}相关的问题。回答要专业但通俗易懂。"),
    ("human", "{question}")
])

print(f"模板变量: {prompt.input_variables}")

In [ ]:
# 使用模板
messages = prompt.format_messages(
    role="Python 导师",
    domain="Python 编程",
    question="什么是装饰器？简单举个例子"
)

response = chat.invoke(messages)

print("角色: Python 导师")
print("问题: 什么是装饰器？")
print(f"\n回复:\n{response.content}")

---
## 2. Chain（链）和 LCEL

### 什么是 LCEL？

**LCEL (LangChain Expression Language)** 使用 `|` 操作符将多个组件连接成一个链：

```
prompt | model | output_parser
```

这类似于 Unix 管道，数据从左到右流动：
1. `prompt` 接收输入，生成提示词
2. `model` 接收提示词，生成回复
3. `output_parser` 处理回复，返回最终结果

In [ ]:
from langchain_core.output_parsers import StrOutputParser

# 创建组件
prompt = ChatPromptTemplate.from_template(
    "将以下内容翻译成{language}：\n\n{text}"
)
model = create_qwen_chat()
output_parser = StrOutputParser()  # 将 AIMessage 转换为字符串

# 使用 LCEL 创建链
chain = prompt | model | output_parser

print("✅ 翻译链创建完成")
print("\n链的结构: prompt → model → output_parser")

In [ ]:
# 调用链
result = chain.invoke({
    "language": "英文",
    "text": "人工智能正在改变我们的生活方式。"
})

print("原文: 人工智能正在改变我们的生活方式。")
print(f"翻译 (英文): {result}")

In [ ]:
# 同一个链，翻译成不同语言
languages = [("日语", "今天天气真好！"), ("法语", "我爱编程"), ("韩语", "谢谢你的帮助")]

for lang, text in languages:
    result = chain.invoke({"language": lang, "text": text})
    print(f"{text} → ({lang}) {result}")

### 多步骤链

可以创建多个链，然后组合它们完成复杂任务。

In [ ]:
# 第一步：生成故事大纲
outline_prompt = ChatPromptTemplate.from_template(
    "请为一个关于{theme}的短故事生成一个简短的大纲（3个要点）。"
)

# 第二步：根据大纲写故事
story_prompt = ChatPromptTemplate.from_template(
    "根据以下大纲，写一个100字左右的短故事：\n\n{outline}"
)

# 创建两个链
outline_chain = outline_prompt | model | StrOutputParser()
story_chain = story_prompt | model | StrOutputParser()

print("✅ 多步骤链创建完成")

In [ ]:
# 执行多步骤任务
theme = "一只勇敢的小猫"
print(f"🐱 主题: {theme}")
print("=" * 50)

# 第一步：生成大纲
print("\n📋 第一步：生成大纲")
outline = outline_chain.invoke({"theme": theme})
print(outline)

# 第二步：根据大纲写故事
print("\n📖 第二步：写故事")
story = story_chain.invoke({"outline": outline})
print(story)

### RunnablePassthrough 和 RunnableParallel

- `RunnablePassthrough`: 原样传递输入
- `RunnableParallel`: 并行执行多个操作

In [ ]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

# 创建一个同时返回原文和翻译的链
translate_prompt = ChatPromptTemplate.from_template(
    "将以下中文翻译成英文，只返回翻译结果：\n{text}"
)

# RunnableParallel 允许并行执行多个操作
chain = RunnableParallel(
    original=RunnablePassthrough(),  # 传递原始输入
    translated=translate_prompt | model | StrOutputParser()  # 翻译
)

result = chain.invoke({"text": "学习编程很有趣！"})

print(f"原文: {result['original']['text']}")
print(f"翻译: {result['translated']}")

---
## 3. 实战：代码审查助手

让我们创建一个实用的代码审查工具。

In [ ]:
# 创建代码审查提示词模板
review_prompt = ChatPromptTemplate.from_messages([
    ("system", """你是一位经验丰富的代码审查专家。
请审查用户提供的代码，从以下几个方面给出建议：
1. 代码质量和可读性
2. 潜在的 bug 或问题
3. 性能优化建议
4. 最佳实践

请用简洁的中文回答。"""),
    ("human", "请审查以下 {language} 代码：\n\n```{language}\n{code}\n```")
])

# 创建审查链
review_chain = review_prompt | model | StrOutputParser()

print("✅ 代码审查助手创建完成")

In [ ]:
# 待审查的代码
test_code = '''
def find_user(users, name):
    for i in range(len(users)):
        if users[i]["name"] == name:
            return users[i]
    return None

def get_user_emails(users):
    emails = []
    for user in users:
        emails.append(user["email"])
    return emails
'''

print("📝 待审查的代码:")
print(test_code)
print("=" * 50)

In [ ]:
# 执行代码审查
review = review_chain.invoke({
    "language": "python",
    "code": test_code
})

print("🔍 审查结果:")
print(review)

---
## 4. 练习

尝试创建自己的链来完成以下任务：

In [ ]:
# 练习 1：创建一个文章摘要链
# 输入一篇文章，输出3句话的摘要

summary_prompt = ChatPromptTemplate.from_template(
    "请用3句话总结以下文章的要点：\n\n{article}"
)

summary_chain = summary_prompt | model | StrOutputParser()

# 测试
article = """
人工智能（AI）正在深刻改变我们的生活和工作方式。从智能手机上的语音助手，到自动驾驶汽车，
再到医疗诊断系统，AI的应用已经渗透到各个领域。随着大语言模型的发展，AI现在能够进行复杂的
对话、写作和编程。然而，这也带来了新的挑战，包括数据隐私、就业影响和伦理问题。专家们认为，
我们需要在推动技术发展的同时，建立相应的监管框架，确保AI的发展造福人类。
"""

summary = summary_chain.invoke({"article": article})
print("📄 文章摘要:")
print(summary)

In [ ]:
# 练习 2：创建一个问答链
# 先给出背景知识，然后回答问题

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个知识问答助手。根据提供的背景知识回答问题，如果背景知识中没有相关信息，请说明。"),
    ("human", "背景知识：\n{context}\n\n问题：{question}")
])

qa_chain = qa_prompt | model | StrOutputParser()

# 测试
context = "LangChain 是一个用于开发 LLM 应用的框架。它提供了 Prompt Template、Chain、Agent 等核心概念。LangGraph 是 LangChain 团队开发的，用于构建有状态的智能体工作流。"

questions = [
    "LangChain 是什么？",
    "LangGraph 是谁开发的？",
    "TensorFlow 是什么？"  # 背景知识中没有的
]

for q in questions:
    answer = qa_chain.invoke({"context": context, "question": q})
    print(f"❓ {q}")
    print(f"💡 {answer}\n")

---
## 📝 本课小结

| 概念 | 说明 |
|------|------|
| `ChatPromptTemplate` | 聊天提示词模板，支持变量替换 |
| `from_template()` | 从字符串创建模板 |
| `from_messages()` | 从消息列表创建模板 |
| `LCEL (\|)` | 使用管道操作符连接组件 |
| `StrOutputParser` | 将 AI 回复转换为字符串 |
| `RunnablePassthrough` | 原样传递输入 |
| `RunnableParallel` | 并行执行多个操作 |

## ➡️ 下一课

下一课我们将学习如何给 AI 添加**工具（Tools）**能力，让它能够获取实时信息、执行计算等操作！